In [1]:
import numpy as np 
import msprime
from scipy import linalg, stats
import utils
import matplotlib.pyplot as plt 
import seaborn as sns 
import pysam
import pandas as pd
from typing import Callable

np.random.seed(42)

# Setup

## Simulate data and clean real data

In [ ]:
# Clean old data

vcf_path = "raw_data/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"
vcf_file = pysam.VariantFile(vcf_path)

records = [record for record in vcf_file.fetch()]


# Filter data

#16 minutes: 154k observations
filtered_by_maf = utils.filter_maf(records)
#2 minutes: no change
filtered_by_missingness = utils.filter_missingness(filtered_by_maf)
#13 seconds: no change
filtered_by_quality = utils.filter_quality(filtered_by_missingness)

records = filtered_by_quality

# Getting the number of samples and SNPs
num_samples = len(vcf_file.header.samples)
num_snps = len(records)

# Create an empty genotype matrix
G = np.empty((num_samples, num_snps), dtype=int)

# Fill the matrix
for j, record in enumerate(records):
    for i, sample in enumerate(vcf_file.header.samples):
        genotype = record.samples[sample].allele_indices
        # Biallelic SNPs
        if genotype == (0, 0):
            G[i][j] = 0
        elif genotype in [(0, 1), (1, 0)]:
            G[i][j] = 1
        else:
            G[i][j] = 2


U, S, Vt = np.linalg.svd(G, full_matrices = False)

np.savez_compressed("../processed_data/real/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes)_SVD.npz", U = U, S = S, Vt = Vt)
np.savez_compressed("../processed_data/real/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.npz", G)

In [ ]:
# We simulate data of various sizes, sequence lengths, and sparsities
sample_sizes = [50, 200, 1000, 10000]
sequence_lengths = [10**x for x in range(4, 8)]
mutation_rates = [1e-4, 1e-6, 1e-8] #lower mutation rate = sparser
recombination_rate = 1e-7

G_simulated = {}

for n in sample_sizes:
    for length in sequence_lengths:
        for mut_rate in mutation_rates:
            G = utils.simulate_diploid_genotypes(n, length, mut_rate, recombination_rate)
            if G.shape[1] >= 10:
                G_simulated[(n, length, mut_rate)] = G

In [ ]:
# Save simulated data
data_dir = "../processed_data/simulated"
for (params, G) in G_simulated.items():
    file_name = f"{params[0]}_{params[1]}_{params[2]}.genotypes.npz"
    np.savez_compressed(f"{data_dir}/{file_name}", G)

## Load Data 

In [ ]:
# Real data from 1000 genomes project
G = np.load("../processed_data/real/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.npz")['arr_0']
svd_data = np.load("../processed_data/real/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes)_SVD.npz")

U, S, Vt = svd_data["U"], svd_data["S"], svd_data["Vt"]

In [2]:
# Load simulated data

sample_sizes = [50, 200, 1000, 10000]
sequence_lengths = [10**x for x in range(4, 8)] 
mutation_rates = [1e-4, 1e-6, 1e-8] #lower mutation rate = sparser
recombination_rate = 1e-7

G_simulated = {}

simulated_dir = "../processed_data/simulated/"
for n in sample_sizes:
    for length in sequence_lengths:
        for mut_rate in mutation_rates:
            file_name = f"{n}_{length}_{mut_rate}.genotypes.npz"
            file_dir = simulated_dir + file_name 
            try:
                G = np.load(file_dir)["arr_0"]
                G_simulated[(n, length, mut_rate)] = G
            except:
                pass

## Summary Statistics

In [ ]:
# Calculate sparsity statistics for G
utils.count_genotypes(G)

In [ ]:
# Calculate sparsity statistics
G_simulated_props = {key: utils.count_genotypes(G_simulated[key]) for key in G_simulated.keys()}

In [ ]:
# Generate dataframe
df_keys = [{'Parameters': key, 'Shape of G': G_simulated[key].shape,
         'Proportion Zeros': value[0], 'Proportion Ones': value[1], 'Proportion Twos': value[2]}
        for key, value in G_simulated_props.items()]

G_df = pd.DataFrame(df_keys)

# Write to LaTeX
with open('../figures/sim_data_table.txt', 'w') as f:
    f.write(G_df.to_latex(index=False,
                          bold_rows=False,
                          float_format="%.2f"))

G_df

# Algorithms

## Darnell et al: ARSVD

Paper: https://www.jmlr.org/papers/volume18/15-143/15-143.pdf 

Implementation: https://github.com/gdarnell/arsvd/blob/master/dimension_reduction.py 


In [3]:
def rsvd(X, dstar, power_iters=2, delta=10):
	""" Perform rsvd algorithm on input matrix.
		Method must be supplied dstar.
		Returns truncated svd (U,S,V).
	Parameters
	----------
	X : int matrix
    	Matrix of n x m integers, where m <= n. If n < m,
    	matrix will be transposed to enforce m <= n.
   	dstar : int
   		The latent (underlying) matrix rank that will be
   		used to truncate the larger dimension (m).
   	power_iters : int
   		default: 2
   		Number of power iterations used (random matrix multiplications)
   	delta : int
   		default: 10
   		oversampling parameter (to improve numerical stability)
    Returns
	-------
	int matrix
    	Matrix of left singular vectors.
    int matrix
    	Matrix of singular values.
    int matrix
    	Matrix of right singular vectors.
    """
	transpose = False 
	if X.shape[0] < X.shape[1]:
		X = X.T 
		transpose = True 

	if power_iters < 1:
		power_iters = 1

	# follows manuscript notation as closely as possible
	P = np.random.randn(X.shape[0],dstar+delta)	
	for i in range(power_iters):
		P = np.dot(X.T,P)
		P = np.dot(X,P)
	Q,R = np.linalg.qr(P)
	B = np.dot(Q.T,X)
	U,S,V = linalg.svd(B)
	U = np.dot(Q, U)

	# Remove extra dimensionality incurred by delta
	U = U[:, 0:dstar]
	S = S[0:dstar]

	return (V.T, S, U.T) if transpose else (U, S, V)


def stabilityMeasure(X, d_max, B=5, power_iters=2):
	""" Calculate stability of 
	Parameters
	----------
	X : int matrix
		input matrix to determine rank of
	d_max : int
		upper bound rank to estimate
	B : int
		default: 5
		number of projections to correlate
	power_iters : int
		default: 2
   		Number of power iterations used (random matrix multiplications)
	Returns
	-------
	int
		Latent (lower-dimensional) matrix rank
	"""
	singular_basis = np.zeros((B,X.shape[0],d_max))
	# calculate singular basis under multiple projections
	for i in range(B):
		U = rsvd(X,d_max)[0]
		singular_basis[i,:,:] = U[:,0:d_max]

	# calculate score for each singular vector
	stability_vec = np.zeros((d_max))
	for k in range(d_max):
		stability = 0
		for i in range(0,B-1):
			for j in range(i+1,B):
				corr = stats.spearmanr(singular_basis[i,:,k],singular_basis[j,:,k])[0]
				stability = stability + abs(corr)
		N = B*(B-1)/2
		stability = stability/N
		stability_vec[k] = stability

	# wilcoxon rank-sum test p-values
	p_vals = np.zeros(d_max-2)
	for k in range(2,d_max):
		p_vals[k-2] = stats.ranksums(stability_vec[0:k-1],stability_vec[k-1:d_max])[1]

	dstar = np.argmin(p_vals)
	
	return dstar

## Saibaba et al: GSVD

Paper: https://www.osti.gov/servlets/purl/1769894 

In [4]:
def gsvd():
    pass 

# Benchmarking

In [13]:
def benchmark(f: Callable, k: int, data: dict, *args):
    """ 
    Benchmarks performance of an algorithm on the given datasets.

    Inputs:
    - f (Callable): the PCA algorithm
        - should take in an argument for data and an argument for number of principal components
    - k (int): the desired number of principal components
    - data (dict): the datasets, where data[params] = dataset

    Returns:
    - dict of accuracy metrics by dataset
    """
    accuracy_metrics = {}

    for (params, dataset) in data.items():
        # Calculate proportion of total variance for top k PCs
        accuracy_key = (params, k)
        pcs = f(dataset, k, *args)
        
        accuracy_metrics[accuracy_key] = utils.compute_prop_variance(pcs, dataset)

    return accuracy_metrics

In [14]:
def center_X(X: np.array):
    """ 
    Returns a centered version of X so that each column has mean 0.

    Input:
    - X (2D numpy array): dataset

    Returns:
    - Xtilde, centered version of X
    """
    return X - np.mean(X, axis = 0)

In [15]:
# Values for k
k_vals = [1, 2, 3, 5, 10, 20, 30]

### ARSVD

In [16]:
def pca_arsvd(X, k, dstar):
    m, n = X.shape[0], X.shape[1]
    transpose = False 
    # Check if transposing is necessary
    if X.shape[0] < X.shape[1]:
        X = X.T 
        transpose = True 
        VT, _, _ = rsvd(X, dstar)
    else:
        _, _, V = rsvd(X, dstar)

    
    # Compute top k pcs
    Xtilde = center_X(X)

    pcs = []

    if transpose:
        for i in range(min(dstar, k)):
            pc = VT[:, i]
            pcs.append(pc)
    else:
        for i in range(min(dstar, k)):
            pc = V[i, :]
            pcs.append(pc)
    
    return np.array(pcs).T

In [17]:
# Benchmarking
arsvd_benchmark = []

for k in k_vals:
    results = benchmark(pca_arsvd, k, G_simulated, 10)
    arsvd_benchmark.append(results)

ValueError: diag requires an array of at least two dimensions

In [ ]:
test 

In [ ]:
test.keys()

In [ ]:
test[((50, 10000, 0.0001), 5)].shape